---
title: "Diff in Diff: From 2x2 to GxT DID"
subtitle: "Advances, Problems and Solutions"
author: Fernando Rios-Avila
institute: "Levy Economics Institute of Bard College"
format: 
    revealjs: 
        slide-number: true
        width: 1700
        height: 900
        code-fold: true
        code-overflow: wrap
        echo: true
        css: styles.css 
bibliography: did_reg.bib        
jupyter: nbstata
nocite: | 
  @*
---


## Why differences in diffences?

- Differences-in-Differences (DID) is one of the most popular tools in applied economics for analyzing causal effects of an intervention or policy treatment.
- Under reasonable assumptions, DID allows you to identify these effects by comparing changes in the treatment group with changes in the control group.
    - This method of identification even allows for controlling for self-selection in the treatment.
- In recent years, several advances in DID have been developed, revealing issues with the traditional DID design, particularly with the application of TWFE.
- Today, I will briefly present the problems and solutions that have been proposed, and how they are implemented in `Stata`.

- So lets Start with the basics: 2x2 DID

## 2x2 DID: Cannonic Design

- In the 2x2 DID design, we have 2 groups:
  - Control ($D=1$) y treated ($D=0$), 
- Which are observed for two periods of time:
  - Before ($T=0$) and after ($T=1$) the treatment.
- For all groups and periods of time, we observe the *realized* outcome $Y$, but cannot observe all *potential* outcomes.
- This setup is valid both for panel data and repeat cross-sections, but will focus on panel data.
   
## Potential outcomes and Treatment Effects

- **Potential outcomes** are the outcomes we would observe for each observation in the data if it were assigned to either the treatement or control group. 
  - $Y_{i,t}(D=0)$: Potential outcome for individual $i$ at time $t$ if he was never treated.
  - $Y_{j,s}(D=1)$: Potential outcome for individual $j$ at time $s$ if he was treated.
- However, we are only able to observe one of this outcomes:
$$Y_{i,t} = Y_{i,t}(1)D_i + Y_{i,t}(0)(1-D_i)$$

- Where $D_i$ is a dummy indicates the effective treatment status of individual $i$.

- Now, we are interested in Treatment Effects for $i$, or any summary measures, after the treatment is applied:

$$TE = Y_{i,1}(1)-Y_{i,1}(0) \text{ or } ATT =E(Y_{i,1}(1)-Y_{i,1}(0)|D=1)$$


## The DID Estimator: Assumptions

- Because we can't observe both potential outcomes, we need to make assumptions to identify the treatment effect.
  
1. **Stable Unit Treatment Value Assumption (SUTVA)**: The treatment status of one unit does not affect the potential outcomes of other units. (No spillovers)

2. **Parallel Trends**: In the absence of treatment, in average, both groups would have followed the same trend (changes) over time.

$$\color{blue}{E(Y_{i,1}(0) - Y_{i,0}(0)|D_i=1)} = \color{green}{E(Y_{i,1}(0)-Y_{i,0}(0)|D_i=0)}$$
   
3. **No Anticipation**: Before the treatment takes place ($T=0$), the potential outcomes do not depend on the treatment status. 

$$Y_{i,0}(0) = Y_{i,0}(1)$$

## The DID Estimator

- Lets take a second look at the ATT definition:
$$\begin{aligned}
ATT &=E(Y_{i,1}(1)|D=1)-\color{red}{E(Y_{i,1}(0)|D=1)} \\
&=E(Y_{i,1}|D=1)-\color{red}{E(Y_{i,1}(0)|D=1)}
\end{aligned}
$$

- The problem is to estimate the **red** component, to obtain an estimator for the ATT. 

- However, under the PTA and No Anticipation, the observed part can be written as:
$$E(Y_{i,1}(0)|D=1) = E(Y_{i,1}-Y_{i,0}|D=0) + E(Y_{i,0}|D=1)
$$

- Which gives the usual ATT estimator:

$$ATT =\underbrace{E(Y_{i,1} - Y_{i,0}|D=1)}_{\Delta treat} - \underbrace{E(Y_{i,1}-Y_{i,0}|D=0)}_{\Delta control}$$ 

## Graphically


In [ ]:
*| echo: false
ssc install did_imputation

In [ ]:
*| echo: false
*| fig-align: center

ssc install color_style
set scheme white2
color_style tableau

set linesize 250
clear
input t d y
0 0 0.5
0 1 0.6
1 0 0.8
1 1 1.1
0 2 0.6
1 2 0.9
end
 
two (scatter y t if d==0, connect(l)) ///
	(scatter y t if d==1, connect(l)) ///
	(scatter y t if d==2, connect(l)) ///
	(pci .9 1 1.1 1, lw(2) color(gs5*.5)), ///
	xscale(range(-.1 1.1)) ///
	legend(order(1 "Control" 2 "Treated" 3 "Counterfactual" 4 "ATT")) ///
    ysize(10) xsize(15)


## How to estimate it?

- **Difference-in-Means**: 
  
$$\widehat{ATT}=\bar Y^{D=1,T=1} - \bar Y^{D=1,T=0} - [ \bar Y^{D=0,T=1} - \bar Y^{D=0,T=0} ]$$

- **Regression**: 

$$Y_{i,t} = \beta_0 + \beta_1 D_i + \beta_2 t + \beta_3 (D_i \times t) + \epsilon_{i,t}$$

- With Panel Data: **Fixed Effects**
  
$$\begin{aligned}
Y_{i,t} &= \beta_0 + \sum \delta_i  + \beta_2 t + \beta_3 (D_i \times t) + \epsilon_{i,t} \\
\Delta Y_{i} &= \beta_2 + \beta_3 D_i + \epsilon_{i} \text{ for } t=0 \\
\end{aligned}
$$

## Extension: Adding Controls

- The DID estimator presented above relies on Unconditional Parallel Trends. It may work if both control and treated groups are similar in all aspects, except for the treatment. But they may not. 
- In this case, we can add controls to the model, to account for differences in the groups.

$$Y_{i,t} = \beta_0 + \beta_1 D_i + \beta_2 t + \beta_3 (D_i \times t) +  X_{it} \gamma + \epsilon_{i,t}$$

- But this would be wrong. Why?

## Problems of Adding Controls

- Simply adding controls imposes assumption of homogenous treatement effects. As described in @santanna_doubly_2020, $\gamma$ may not be the same as the ATT. 
  - Solution: Interactions between controls and all group\\time dummies:
$$y_{i,t} = \beta_0 + \sum \delta_i  + \beta_2 t + \beta_3 (D_i \times t) +  X_{it} \gamma + \sum \gamma_x (X-\bar X_{D\times T})*D*T + \epsilon_{i,t}$$

- Controlling for time varying variables may introduce biases (bad controls)
  - Solution: use only pre-treatment variables as controls. (panel) or variables that should not be affected by the treatment (cross-sections)
  - Also: If using Panel Data, one can add as controls all variables history. (see for example @caetano_difference_2022)

## Potential Solution: @santanna_doubly_2020

- DID is a straighforward estimator, that relies strongly on the Unconditional Parallel Trends Assumption. (UPTA)

- One solution to the problem is to use the **Conditional Parallel Trends Assumption** (CPTA), which can be less restrictive.

- This states that PTA holds, only after conditioning on a set of variables $X$. (looking at subgroups).

$$\color{blue}{E(Y_{i,1}(0) - Y_{i,0}(0)|D_i=1,X)} = \color{green}{E(Y_{i,1}(0)-Y_{i,0}(0)|D_i=0,X)}$$

- So, if you can "control" for individual characteristics, you could estimate the treatment effect, and still report Average Treatment Effects for the population.

$$ATT = E\big[ [E(Y_{i,1}(0) - Y_{i,0}(0)|D_i=1,X ] \big]$$

## Added Assumption

- When accouting for covariates, one needs to add an additional assumption to the data:

- There is an overlap in the distribution of $X$ between the treated and control groups. (common support)

$$0 << Pr(D=1|X) << 1$$

- This is important, because if there is no overlap, the treatment effect cannot be identified for all groups, and may create distorted estimates.

## Potential Solution: `drdid`, `teffects` 

- @santanna_doubly_2020 propose various alternatives to estimate the treatment effect with controls, under CPTA. They contrast the implementation of methods that use linear regressions, Inverse Probability Weighting (IPW) and Doubly Robust Estimators (DR).
- Similar estimates could be obtained using `teffects` in Stata.
- But how does this work? Two Cases: 
  - Panel Data: This simplifies the problem, because we can use the Data structure to estimate the model.
  - Crossection: This is more complicated because requires careful consideration of idenfication assumptions, and groups of interest.

## Panel Data 

- Step 1: Estimate the changes in the outcome $\Delta Y_i = Y_{i,1} - Y_{i,0}$. (the First D)
  - Because of this, we are implicitly reducing our sample size: from $N$ to $N/2$ (because of the unestimated fixed effects)
- Step 2: Estimate the treatment effect:
  - Simplest case, no controls:
$$\Delta Y_i = \beta_0 + \beta_1 D_i + \varepsilon_i$$ 
  
- In other words, the model is now a simple linear regression, with 1 period.

## Panel Data: Adding controls

- Because you have now only 1 period, and two groups. All treatment effect estimators can be used.
  - regression outcome, IPW, doubly robust, matching, etc.
  - We use them to estimate the second D of DID
- Empirically: With Transformed data ($\Delta y$) as dependent variable, any of the methods in `teffects` can be used to estimate the treatment effect.
- With untransformed data (RC), you can use `drdid` to estimate the treatment effect, and their Standard errors.
    - You need to make sure you have only 2 periods of data.

## Panel Data: Methods

- **Regression**: reg (`drdid`) or ra (`teffects`) 
    - using $\Delta y$ as dependent variable, estimate a linear model with covariates (No FE) for the untreated, and predict outcomes. This would become your "Potential outcome change under no-treatment"
  
$$ ATT = E(\Delta Y_{i}|D=1) - E({\hat\gamma X|D=1})$$

- **Inverse Probability Weighting**: stdipw (`drdid`) or ipw (`teffects`)
    - Estimate the likelihood of being treated, and use it to identify a propensity score and IPW weights $p(D=1|X) = \hat p(X)$
  
$$ ATT = E(\Delta Y_{i}|D=1) - E\left({\Delta Y_{i} \frac{p(X)}{1-p(X)}|D=0}\right)$$

## Panel Data: Methods

- **Doubly Robust Estimation**: dript and dripw (`drdid`) or ipwra aipw (`teffects`)
    - Estimate the likelihood of being treated, and use it to identify a propensity score and IPW weights $p(D=1|X) = \hat p(X)$
    - Estimate a weighted regression with $\Delta y$ as dependent variable or estimate a weighted regression correction. (see [here](https://friosavila.github.io/app_metrics/app_metrics2.html) slide 17-18) for more details.
    - Combinations of modeling outcome and modeling likelihood and pscores makes the estimator doubly robust.

- For Repeated crossection, the math becomes more complicated to follow and identify the "group of interest"  
- See [here](https://friosavila.github.io/app_metrics/app_metrics2.html) slide 53-54 for more details.
  
## Example {.scrollable}

Some Data Preparation


In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false
qui:frause lalonde, clear
keep  if treated==0 | sample==2
bysort id (year):gen dy = re[2]-re[1]

Using `teffects`:


In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false

teffects ra (dy educ black married nodegree hisp re74) (experimental) if year==1975 , atet

Using `drdid`:


In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false
 
drdid re educ black married nodegree hisp re74 , ivar(id) time(year) tr(experimental) reg

by hand:


In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false
qui:reg dy educ black married nodegree hisp re74 if exper==0
predict dy_hat, xb
gen att_i = dy-dy_hat
tabstat att_i, by(exper) 

## Highlights I: Identification

- The identification relies on the non-anticipation assumption and the parallel trends assumption.
  - Neither of these assumptions can be tested, but they can be supported by the data.
- For Implementation, If one has panel data, `teffects` and `drdid` can be used to estimate the treatment effect, and their standard errors. (after preparing the data)
- Otherwise, `drdid` can be used for repeated crossection as well. (or GMM)

## Highlighs II: Overlapping  

- The **overlapping** assumption is important, and has different implications for the different methods.
- When the estimation method is `reg` or `ra`, the overlapping assumption is less biding, because the model is extrapolates to created "potential outcomes" for the treated group.
  - However, extrapolation may not be accurate, or create incorrect extrapolations (base category dummies)
- With doubly robust methods, the overlapping assumption is more relevant. `dript` is particularly sensitive to this assumption, and may not even converge if there is weak overlap.
- For most practical purposes, `dripw` may be the most stable, and preferred method.

## Highlighs III: Controls

- With panel data, `drdid` automatically constrains model specification to use pre-treatment information only.
  - Added work is required to include post-treatment information.
- With Repeated crossection, `drdid` would use both pre-treatment and post-treatment information.
  
# From 2x2 to GxT DID

## How GxT DID was suppoused to work


- While the 2x2 DID design is simple to understand, it does not reflect the type of information that is available in most cases. (G>2 and T>2)
 
- Until few years ago, when this kind of data was available, the standard approach was to use a generalized DID design, which extended the use of Fixed Effects:

$$
\begin{aligned}
2\times2: Y_{i,t} &= \beta_0 + \beta_1 T  + \beta_2 D_i + \beta_3 (D_i \times T) + \epsilon_{i,t} \\
G\times T: Y_{i,t} &= \sum \gamma_t + \sum \delta_i  + \theta^{fe} PT_{i,t} + \epsilon_{i,t} \\
\end{aligned}
$$

where $PT_{i,t}$ assume the value of 1 for the treated group After the treatment is applied, and 0 otherwise, and $\theta^{fe}$ representing the treatment effect.

- Little that we know that this approach only works if the treatment effect is homogeneous across all groups and periods.
  
## Why it doesn't work? 

There are at least two-ways that have been used to explain why the Standard TWFE does not work:

- **The "Bad Controls" Problem**: @goodmanbacon2021 shows that the TWFE estimator for ATT is a kind of weighted average of all possible DID one could estimate. Some of these would not be **good** DID designs.
- **Negative Weights**: @dechaisemartin2020 and @borusyak2023revisiting show that because TWFE ATT estimator is a weighted average of all group-Specific ATT, some may include negative weights.
  
  This may produce negative ATT even if all group-specific ATT are positive. 

- However, when the treatment effect is homogeneous, Neither of these situations would be a problem.

## The "Bad Controls" Problem


In [ ]:
*| echo: false
*| output: false

clear
set obs 4
gen id = _n
gen g = 0 in 1
replace g = 4 in 2
replace g = 7 in 3
replace g = 11 in 4
gen y0 = 0+(id-3)*0.1
expand 10
bysort id:gen t = _n
bysort id:gen y = y0 + id*(t>=g)

two (scatter y t if id==1, connect(l)) ///
	(scatter y t if id==2, connect(l)) ///
	(scatter y t if id==3, connect(l)) ///	
	(scatter y t if id==4, connect(l)) , ///
	xscale(range(0 11)) xlabel(1/10) ///
	legend(order(1 "Always Treated" 2 "Early treated" 3 "Late Treated" 4 "Never Treated"))

graph export fig1.png, width(1000)	replace


two (scatter y t if id==1, connect(l)) ///
	(scatter y t if id==2, connect(l)) ///
	(scatter y t if id==3, connect(l)) ///	
	(scatter y t if id==4, connect(l)) ///
	(scatter y t if id==2 & inrange(t,1,6), connect(l) lw(1) pstyle(p2)) ///
	(scatter y t if id==3 & inrange(t,1,6), connect(l) lw(1) pstyle(p3)) , ///
	xscale(range(0 11)) xlabel(1/10) ///
	legend(order(1 "Always Treated" 2 "Early treated" 3 "Late Treated" 4 "Never Treated"))	
	
graph export fig2.png, width(1000)	replace
	
two (scatter y t if id==1, connect(l)) ///
	(scatter y t if id==2, connect(l)) ///
	(scatter y t if id==3, connect(l)) ///	
	(scatter y t if id==4, connect(l)) ///
	(scatter y t if id==2 & inrange(t,4,10), connect(l) lw(1) pstyle(p2)) ///
	(scatter y t if id==3 & inrange(t,4,10), connect(l) lw(1) pstyle(p3)) , ///
	xscale(range(0 11)) xlabel(1/10) ///
	legend(order(1 "Always Treated" 2 "Early treated" 3 "Late Treated" 4 "Never Treated"))	
graph export fig3.png, width(1000)	replace

:::{.panel-tabset}

## All Cases

![](fig1.png){height=50%}

## Early Treated vs Later Treated (Good)

![](fig2.png)

## Early Treated vs Later Treated (bad)

![](fig3.png)

:::

## The "Bad Controls" Problem


In [ ]:
*| echo: false
*| output: false
	
	
clear
set seed 101
set obs 4
gen id = _n
gen g = 0 in 1
replace g = 4 in 2
replace g = 7 in 3
replace g = 11 in 4
gen y0 = 0+(id-3)*0.1
gen slp = runiform(-.1,0.1)
expand 10
bysort id:gen t = _n
bysort id:gen y = y0 + 0.2*id*(t>=g) + (t>=g)*(t-g)*slp

two (scatter y t if id==1, connect(l)) ///
	(scatter y t if id==2, connect(l)) ///
	(scatter y t if id==3, connect(l)) ///	
	(scatter y t if id==4, connect(l)) , ///
	xscale(range(0 11)) xlabel(1/10) ///
	legend(order(1 "Always Treated" 2 "Early treated" 3 "Late Treated" 4 "Never Treated"))

graph export fig1b.png, width(1000)	replace


two (scatter y t if id==1, connect(l)) ///
	(scatter y t if id==2, connect(l)) ///
	(scatter y t if id==3, connect(l)) ///	
	(scatter y t if id==4, connect(l)) ///
	(scatter y t if id==2 & inrange(t,1,6), connect(l) lw(1) pstyle(p2)) ///
	(scatter y t if id==3 & inrange(t,1,6), connect(l) lw(1) pstyle(p3)) , ///
	xscale(range(0 11)) xlabel(1/10) ///
	legend(order(1 "Always Treated" 2 "Early treated" 3 "Late Treated" 4 "Never Treated"))	
	
graph export fig2b.png, width(1000)	replace
	
two (scatter y t if id==1, connect(l)) ///
	(scatter y t if id==2, connect(l)) ///
	(scatter y t if id==3, connect(l)) ///	
	(scatter y t if id==4, connect(l)) ///
	(scatter y t if id==2 & inrange(t,4,10), connect(l) lw(1) pstyle(p2)) ///
	(scatter y t if id==3 & inrange(t,4,10), connect(l) lw(1) pstyle(p3)) , ///
	xscale(range(0 11)) xlabel(1/10) ///
	legend(order(1 "Always Treated" 2 "Early treated" 3 "Late Treated" 4 "Never Treated"))	
graph export fig3b.png, width(1000)	replace	

:::{.panel-tabset}

## All Cases

![](fig1b.png)

## Early Treated vs Later Treated (Good)

![](fig2b.png)

## Early Treated vs Later Treated (bad)

![](fig3b.png)

:::

## Negative Weights

- To understand the idea of Negative Weights, lets consider the following example:

$$Y_{i,t} = \sum \delta_i + \sum \gamma_t  + \theta PT + \epsilon_{i,t}$$

If the panel data is balanced, and we apply FWL, we can use partialling out the Fixed Effects:

$$\widetilde{PT}_{it}=PT_{it}-\overline{PT}_{i}-\overline{PT}_{t}+\overline{PT}$$

and estimate $\theta^{fe}$ as:

$$\hat\theta^{fe} = \frac{\sum \widetilde{PT_{it}} Y_{it}}{\sum \widetilde{PT}_{it}^2}
= w_{it} Y_{it}
$$

##

$$\widetilde{PT}_{it}=PT_{it}-\overline{PT}_{i}-\overline{PT}_{t}+\overline{PT}$$

- $\overline{PT}_{i}$ is the share of periods a unit is observed to be treated. (larger for early treated)
- $\overline{PT}_{t}$ is the share of units treated at time $t$. (increasing)
- $\overline{PT}$ Share of treated "unit-periods"

Thus,

- Units that are treated early, (high $\overline{PT}_{i}$)
- but are analyzed at later periods ($\overline{PT}_{t}$ increasing in $t$)

Will be more likely to have a negative weight $w_{it}$, thus contaminating the ATT estimate.

## Graphically


In [ ]:
*| echo: false
*| fig-align: center
*| output: false
clear
set obs 100
gen id = _n
gen gvar = runiformint(1,10)
expand 10
bysort id:gen t =_n

gen trt = t>=gvar

reghdfe trt , abs(id t) resid
bysort gvar t:gen flag = _n

In [ ]:
*| echo: false
*| fig-align: center

two (scatter _reghdfe_resid t if trt==1 & gvar==1 & flag==1, connect(l)) ///
	(scatter _reghdfe_resid t if trt==1 & gvar==2 & flag==1, connect(l)) ///
	(scatter _reghdfe_resid t if trt==1 & gvar==5 & flag==1, connect(l)) ///
	(scatter _reghdfe_resid t if trt==1 & gvar==6 & flag==1, connect(l)) ///
	(scatter _reghdfe_resid t if trt==1 & gvar==9 & flag==1, connect(l)) ///
	(scatter _reghdfe_resid t if trt==1 & gvar==10 & flag==1, connect(l)) , ///
	xscale(range(0 11)) ///
	legend(order(1 "Always Treated" 2 "Early Treated" 5 "Late Treated" 6 "Latest Treated"))


## Summarizing the Problem

- Both Negative Weights and Bad Controls are capturing the same problem.
  
1. By using already treated units as controls (bad controls), we are implicitly applying negative weights to units that are already treated.

2. This would not be a problem on its own, if the treatment effect is homogeneous across all groups and periods.

3. However, if the treatment effect is heterogeneous, Parallel Trends between already treated and late treated units may not hold. 

## GxT DID: Generalized DID
### Setup
- In the Generalized DID
  - One has access to multiple periods of data: $t = 1,2,...,T$.
  - And units can be treated at any point before, during or after the available Data $G = -k,..1,2,..,T+l$.
    - This is the cohort or group.
  - We also assume that once a unit is treated, it remains treated. (no reversals/no cohort-change)

- From our perspective, units treated **at or before** $t=1$ will be considered as **Allways treated**.
  - This units cannot be used for analysis.
- For any practical purpose, if we do not observe a unit being treated in the window of time observed, we assume its **Never Treated**. 
  - For Notation we say these units belong to $g=\infty$ (or that they could be treated at some point in the far future)

## GxT DID: Generalized DID
### Potential Outcomes

- In contrast with previous setup, with the GDID, we believe observations have multiple potential outcomes, for each period of time.
	- $Y_{i,t}(G)$ is the potenital outcome for individual $i$ at time $t$, if this unit would be treated at time $G$. 
	- Thus we state that depending on "when" a unit is treated, the potential outcomes could be different.
  	- This is what it means allowing for heterogeneity.
  
## GxT DID: Generalized DID
### Parallel Trends Assumption

- PTA assumption is also slightly modified. Because we can differentiate between **never-treated** and **not-yet-treated**, one could impose those PTA assumptions.
  
  Never Treated
	$$E(Y_{i,t}(\infty) - Y_{i,t-s}(\infty)|G=g) = E(Y_{i,t}(\infty) - Y_{i,t-s}(\infty)|G=\infty) \forall s>0 $$

  Not Yet Treated

  	$$E(Y_{i,t}(\infty) - Y_{i,t-s}(\infty)|G=g) = E(Y_{i,t}(\infty) - Y_{i,t-s}(\infty)|G=g') \forall s>0 $$

- Which suggests PTA hold for all pre- and post-treatment periods. 
- Some methods only rely on Post-treatment PTA.
  
## GxT DID: Generalized DID
### No Anticipation

- As before, we also require no anticipation.
  - That the before treatment takes place (or is announced), the potential outcomes do not depend on the treatment status.

$$Y_{i,t}(G) = Y_{i,t}(\infty) \text{ if } t<G $$ 

- Also important.
  - $E(Y_{i,t}(G)|g=G) = E(Y_{i,t}|g=G)$ if $t>G$ 
  - $E(Y_{i,t}(\infty)|g=\infty) = E(Y_{i,t}|g=\infty)$

# Solutions

## General Overview

The root of the problem with TWFE: 

> If treatment effects are heterogenous, the TWFE estimator of treatment effects produces incorrect estimates
> because of "negative weights" or "bad controls"

To solve the problem we need to do (at least) one of three things:

  1) Avoid using bad controls (@borusyak2023revisiting and @gardner2022twostage)
  2) Allow for heterogeneity in the treatment effect. (@wooldridge_2021 and @sun_estimating_2021)
  3) Use *only* good DID designs. (@callaway_2021)

## Avoid using bad controls: `did_imputation` and `did2s`
#### @borusyak2023revisiting and @gardner2022twostage

- Both methods are based on the idea of "imputing" the missing potential outcomes $Y_{it}(0)$ for the treated group, using a method similar to the one used for 2x2 DID. A two-stage approach.
  
1. We know that estimating the following model would be incorrect:

$$y_{it} = \delta_i + \gamma_t + \theta D_{it} + \epsilon_{it}$$

2. However, what this authors suggest is to identify $\delta_i$ and $\gamma_t$ using pre-treatment data only:

$$y_{it} = \delta_i + \gamma_t + \epsilon_{it} \text{ if } t<g $$

This helps identifying the fixed effects, without any contamination. (its a model for the potential outcome of no treatment)

## Avoid using bad controls: `did_imputation` and `did2s`
#### @borusyak2023revisiting and @gardner2022twostage

3. Use the previous model to re-estimate 1:

$$y_{it} = \hat \delta_i + \hat \gamma_t + \theta D_{it} + \epsilon_{it}$$

4. In fact, one could use many other specifications to identify Treatment effects, including by group, by calendar, or dynamic effects.

$$y_{it} = \hat \delta_i + \hat \gamma_t + \theta D_{it}*(other Heterogeneity) + \epsilon_{it}$$

## Avoid using bad controls: `did_imputation` and `did2s`
#### @borusyak2023revisiting and @gardner2022twostage

- The identification of TE using the imputation method relies on the same assumptions as the traditional DID design.
- However, It also requires the Parallel Trends Assumption (PTA) to hold for all pre-treatment periods.
  
  - This why how we can use pre-treatment information to predict the potential outcomes for the treated group.
    - At the extreme, one could even identify TE without access to non-treated units!
  
- This also means that the method is sensitive to problems with **long** PTA. Although it can be somewhat relaxed (based on model specification).

- However, its more efficient than other models because it uses all-pretreatment data for estimation.
  - Data requirements are similar to the traditional DID design.

## Example {.scrollable}

- There are two implementations of the method in Stata: `did_imputation` and `did2s`. In addition to the original GMM estimator described in @gardner2022twostage.
- However, the most flexible and Robust implementation is `did_imputation`.


In [ ]:
*| echo: true
*| classes: larger
*| code-fold: false
use http://pped.org/bacon_example.dta, clear
**Estimate model for Pre-treatment data
qui:reghdfe asmrs if post==0, abs(fe1 = stfips fe2 = year)
** Extra polate
bysort stfips (fe1):replace fe1=fe1[1]
bysort year (fe2):replace fe2=fe2[1]

** get TE for i
gen te = asmrs-fe1-fe2-_b[_cons]
sum te if post==0
sum te if post==1

using `did_imputation`:


In [ ]:
*| echo: true
*| classes: larger
*| code-fold: false
** Gvar
egen gvar = csgvar(post), ivar(stfips) tvar(year)
** event
gen event = year - gvar if gvar!=0
** Gvar = . (missing implies never treated)
clonevar gvar2=gvar if gvar!=0
did_imputation asmrs stfips year gvar2, autosample
did_imputation asmrs stfips year gvar2, autosample horizon(1/10) pretrends(5)

## Allow for Heterogeneity: `jwdid` and `eventstudyinteract`
#### @wooldridge_2021 and @sun_estimating_2021

- The work by @wooldridge_2021 suggested that the GDID-TWFE estimator was not a problem perse.
- The problem was that the GDID-TWFE estimator was simply misspecified.

- Instead of modeling:
$$Y_{i,t} = \delta_i + \gamma_t  + \theta^{fe} PT_{i,t} + \epsilon_{i,t}$$

- One should allow for a full set of interactions between the group and time dummies:

$$Y_{i,t} = \delta_i + \gamma_t  + \sum_{g=2}^T \sum_{t=g}^T \theta_{g,t} \mathbb{1}(G=g,T=t) + \epsilon_{i,t}$$

- In this framework, each $\theta_{g,t}$ represents the ATT for each group at a particular period.

## Allow for Heterogeneity: `jwdid` and `eventstudyinteract`
#### @wooldridge_2021 and @sun_estimating_2021

- In the basic setup, this approach is basically the same as the method proposed by @borusyak2023revisiting and @gardner2022twostage.
- Wooldridge, however, was not the first approach that aim to "allow for heterogeneity" in the treatment effect. Early attempts were done by using a dynamic events structure, using both leads ands lags of the treatment variable.
$$Y_{i,t} = \delta_i + \gamma_t  + \sum_{e=-k}^{-2} \theta_e \mathbb{1}(t-G_i=e) + \sum_{e=0}^L \theta_e \mathbb{1}(t-G_i=e) + \epsilon_{i,t}$$

- This not only allows for heterogenous effects across time, but also allows you to analyze pre-treatments effects.

- However @sun_estimating_2021 showed that this approach could also be wrong, if dynamic effects are also heterogenous across groups.

## Allow for Heterogeneity: `jwdid` and `eventstudyinteract`
#### @wooldridge_2021 and @sun_estimating_2021

- As solution, @sun_estimating_2021 propose to use a full set of interactions between the group dummies and the event-study dummies. This is similar to @wooldridge_2021.
$$Y_{i,t} = \delta_i + \gamma_t  + \sum_{g=2}^T \sum_{e=-k}^{-2} \theta_{g,e} \mathbb{1}(t-G_i=e, G=g) + \sum_{g=2}^T \sum_{e=0}^L \theta_{g,e} \mathbb{1}(t-G_i=e, G=g) + \epsilon_{i,t}$$

- In fact, if we write the "event" as "time", it would look very similar to the model proposed by @wooldridge_2021.

$$Y_{i,t} = \delta_i + \gamma_t  + \sum_{g=2}^T \sum_{t=1}^{g-2} \theta_{g,t} \mathbb{1}(T=t, G=g) 
+ \sum_{g=2}^T \sum_{t=g}^{T} \theta_{g,t} \mathbb{1}(T=t, G=g) + \epsilon_{i,t}
$$

- Thus, both approaches are identical if we allow for Full interactions before and after treatment.

## Allow for Heterogeneity: `jwdid` and `eventstudyinteract`
### Assumptions and Limitations

- The model proposed by @wooldridge_2021 follows the same assumptions as @borusyak2023revisiting. (Long PTA)
- @sun_estimating_2021 and the modified @wooldridge_2021, however, only requires PTA to hold "after" treatment takes place.
- Both methods require careful consideration of covariates (time constant), and they require additional work for adding it into a model (variable shifting).
  - The limitations of sample size are somewhat more evident in this framework.
- Wooldridge's approach, however, could also be used beyond the linear case as shown in @wooldridge_2023.
  
## Implementation {.scrollable}

There are various commands that implement @sun_estimating_2021 estimator, including her original command `eventsudyinteract`, as well as `xtevent`.

For @wooldridge_2021, there is now the official Stata18 command `xthdidregress twfe`, the one I developed: `jwdid` and a newer one `wooldid`.

Will focus on `jwdid`.

Base Estimation:


In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false

qui:ssc install frause, replace
frause mpdta, clear
** 
jwdid lemp, ivar(countyreal) tvar(year) gvar(first_treat) 
estat simple

Adding controls

In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false
jwdid lemp lpop, ivar(countyreal) tvar(year) gvar(first_treat) 
estat simple

Compared to `did_imputation`

In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false

gen first2=first_treat if first_treat>0
did_imputation lemp countyreal year first2

In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false
qui:jwdid lemp, ivar(countyreal) tvar(year) gvar(first_treat) 
estat simple

Estimating an effect similar to @sun_estimating_2021


In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false
qui:jwdid lemp, ivar(countyreal) tvar(year) gvar(first_treat) ///
	never //<- request full interaction
estat event

It is also possible to add further restrictions to "event/calendar/group" aggregates


In [ ]:
*| classes: larger
*| echo: true
*| code-fold: false
** requires latest JWDID
qui:net install jwdid, from(https://friosavila.github.io/stpackages) replace
program drop _all
gen subsample = inrange(__event__,2,6)
estat event, other(subsample)

## Using Good DID Designs: `csdid` and `csdid2`
#### @callaway_2021

- A third, and last, approach has been proposed by @callaway_2021.
- In contrast with previous methods (which focus on global estimators), this approach suggests deconstructing the estimation into smaller, but well define pieces: The ATTGTs for 2x2 DIDs

  - Makes it easy to use using **GOOD** DID designs only 
  - and, once all ATTGT's are estimated, they can be aggregated in various ways

- This approach takes full advantage of the fact we know quite well (`drdid` + others) how to estimate 2x2 DID. 
- and (or but) forces you to utilize time constant or pretreatment controls only.

- However, one now needs to estimate as many as periods and cohorts are available in the data.

## Using Good DID Designs: `csdid` and `csdid2`
#### @callaway_2021

- The proposed estimator starts with the assumption that one is interested in the ATTGT:

$$ATT(g,t) = E(y_{i,t}(G) - y_{i,t}(\infty) |G_i =g)
$$

- As in the simpler 2x2 case, however, we cannot observe the $y_{i,t}(\infty)$ after they are treated.

- What @callaway_2021 does is to impute this piece applyint PTA and no Anticipation.
- 
$${E(y_{i,t}(\infty)|G_i =g)} \approx E(y_{i,G-k}|G_i =g) + E(y_{i,t}-y_{i,G-k}| G_i \in Control)
$$

- Thus, the estimator for the ATT(g,t) becomes:
$$\widehat{ATT(g,t)} = E(y_{i,t}- y_{i,G-k}|G_i =g)- E(y_{i,t}- y_{i,G-k}|G_i \in Control)
$$

## Using Good DID Designs: `csdid` and `csdid2`
#### @callaway_2021

$$\widehat{ATT(g,t)} = E(y_{i,t}- y_{i,G-k}|G_i =g)- E(y_{i,t}- y_{i,G-k}|G_i \in Control)$$

1. With no anticipation and PTA, *ANY* period before treatment ($G-k$) could be used to construct the ATT(g,t). 
2. Because of this, it only relies on Post-treatment PTA. (Violations of PTA before treatment have no impact on the estimates)
3. Depending on the analysis of interest, one could choose different "control groups"
   - Never treated: Most common
   - Not-yet-treated: Include observations that up to time $t$ have not been treated.
   - For pre-treatment ATT(g,t)'s, the Not-yet treated cound include all cohorts not treated until $t$ `R` or those not treat until $t$ nor $g$ `Stata`.

## Using Good DID Designs: `csdid` and `csdid2`
#### @callaway_2021

- This approach is relatively easy to implement. The main difficulty is keeping track of ALL the ATT(g,t)s that are estimated (and their VCV)
- However, once all ATT(g,t)'s are obtained, aggregation is straight forward:

$$
AGG(ATT(g,t)) = \frac{\sum_{g,t \in G\times T}   ATT(g,t) * w_{g,t} * sel_{g,t} }
{\sum_{g,t \in G\times T} w_{g,t} * sel_{g,t} }
$$

- where $w_{g,t}$ represents the size of cohort $g$ at time $t$ used in the estimation of ATT(g,t). 

- and  $sel_{g,t}$ is an indicator for whether that ATTGT will be used in the aggregation.

## Using Good DID Designs: `csdid` and `csdid2`
#### @callaway_2021

Typical Aggregations:

- Simple: $sel_{g,t}=1$ if $t>=g$
- Group/cohort: $sel_{g,t}=1$ if $t>=g$ and $g=G$
- Calendar: $sel_{g,t}=1$ if $t>=g$ and $t=T$
- Event: $sel_{g,t}=1$ if $t-g=e$
- Cevent: $sel_{g,t}=1$ if $t-g \in [ll,\dots,uu]$

and may be possible to combine some of these restrictions

- Also, because the method is based on 2x2 DID, it allows to easily implement various methodologies, including the Doubly Robust.

## Map: How @callaway_2021 relates to @wooldridge_2021 and @borusyak2023revisiting

Relations: Simple case of no covariates

- `jwdid` =  `did_imputation`: `jwdid` can be applied to nonlinear models, but `did_imputation` is more flexible for linear models.
- `jwdid` = `did_imputation`  = `csdid, notyet`: When using the not-yet treated as control, `csdid` will produce the same estimates as the others if there is only **1 pretreatment** period.
- `jwdid, never` = `csdid` = `eventinteract`: If `jwdid` uses full interactions for pre and post treatment periods, the results will be the same as `csdid` or `eventinteract`

Thus the main differences across models is:

- How they use "pre-treatment" information for the estimation of ATTs (long vs short)
- Whether they use only "never-treated" or "not-yet-treated" units for estimation
- And how are covariates treated

## Example {.scrollable}

- `csdid` is the older command, with all functions well documented (helpfile).

- However, it can be slow in large tasks, because of the bottle neck of using the Full Dataset for every task. 

- `csdid2` works almost identically to `csdid`, its faster, but some functions are not yet documented (helpfile is still missing). Will use this one here


In [ ]:
*| echo: true
*| classes: larger
*| code-fold: false
qui:net install csdid2, from(https://friosavila.github.io/stpackages) replace
use http://pped.org/bacon_example.dta, clear

** create Gvar: still needs csdid
egen gvar = csgvar(post), ivar(stfips) tvar(year)

csdid2 asmrs, ///
	ivar(stfips) /// <- required for panel Data. Otherwise RC
	tvar(year)   /// <- Time variable. Should be continuously (month 11, 12, 13, ...)
	gvar(gvar)   
** Default uses "never treated", and produces Long gaps.

- `csdid2` is creating the full set of ATT(g,t) estimations for the data. But will produce no result
- To obtain results and plotting, one must use post estimation commands:


In [ ]:
*| echo: true
*| classes: larger
*| code-fold: false

estat event, /// request event-type estimates
	noavg    /// asks not to produce Averages
	revent(-10/10) // requests to limit the output to events between -10/10
** Its also possible to restrict to specific groups/cohorts rgroup( list of numbers)
** or restrict to specific years rcalendar( list of numbers)

`estat plot`, will produce a figure from the last estimation


In [ ]:
*| echo: true
*| classes: larger
*| code-fold: false
*| output: false
estat plot, xsize(10) ysize(8)
graph export figev.png, replace width(800)

![a](figev.png)
<img src="figev.png" alt="Trulli" style="width:1%">

Advantage of `csdid2` over other methods. Uniform Confidence Intervals using Wildbootstrap SE.

## Conclusions

- In this workshop I aimed to provide a brief overview of the problems with TWFE and the potential solutions.
- 3 main solutions were presented:
  1. Avoid using bad controls (@borusyak2023revisiting and @gardner2022twostage)
  2. Allow for heterogeneity in the treatment effect. (@wooldridge_2021 and @sun_estimating_2021)
  3. Use *only* good DID designs. (@callaway_2021)

## Conclusions

- All methods have their own advantages and disadvantages.
- 3) is the most robust based on the assumptions, but is the least efficient because of the sample size requirements. It also allows for flexiblity of DR estimators, and ensures you use pre-treatment controls only.
- 1) and 2) are more efficient, (more data is used), but rely strongy on Long PTA assumption.
- 2) can be applied in non-linear settings, using nonlinear models. But aggregations are slower to obtain.
- 1) and 2) can also be adapted to consider -dose-response- effects. As well as Treatment Reversals.

  
# Thank you!
### Comments? Questions?


## Suggested readings

::: {#refs}
::: 
